In [1]:
import numpy as np
import sympy as sy
import matplotlib.pyplot as plt
import simtk.openmm as mm
import simtk.unit as unit
import simtk.openmm.app as app
from tqdm import tqdm
import random

In [2]:
n_particles = 1
system = mm.System()
mass   = 14.007 * unit.amu # masa del átomo N
system.addParticle(mass)

force = mm.CustomExternalForce('Eo*(C*x^4-x^2)+M*x + (K/2.0)*(y^2+z^2)')
force.addGlobalParameter('K', 35 * unit.kilocalories_per_mole/unit.angstrom**2)
force.addGlobalParameter('Eo', 6.0 * unit.kilocalories_per_mole/unit.angstrom**2)
force.addGlobalParameter('C', 0.4 * 1/unit.angstrom**2)
force.addGlobalParameter('M', 0.25 * unit.kilocalories_per_mole/unit.angstrom)
force.addParticle(0, [])
system.addForce(force)

0

In [3]:
reps = []
reps.append(int(input("Escribe T inicial:")))
reps.append(int(input("Escribe la temperatura máxima:")))

reps = list(range(reps[0],reps[1]+1))*unit.kelvin
kB = unit.BOLTZMANN_CONSTANT_kB * unit.AVOGADRO_CONSTANT_NA
pressure    = None

num_steps       = 20000
saving_period   = 100
num_steps_saved = int(num_steps/saving_period)

friction   = 1.0/unit.picosecond

platform_name = 'CPU'  
platform = mm.Platform.getPlatformByName(platform_name)

step_size = 0.01*unit.picoseconds

intg = {}
for ii in range(len(reps)):
    for T in reps:
        intg["integrator{0}".format(ii)] = mm.LangevinIntegrator(T, friction, step_size)
    
ts = {}
pos = {}
vel = {}
pot_en = {}
kin_en = {}
init_pos = {}
init_vel = {}
contxt = {}
sts = {}

for ii in range(len(reps)):
    ts["times{0}".format(ii)] = unit.Quantity(np.zeros([num_steps_saved], np.float32), unit.picoseconds)
    pos["positions{0}".format(ii)]  = unit.Quantity(np.zeros([num_steps_saved,3], np.float32), unit.angstroms)
    vel["velocities{0}".format(ii)] = unit.Quantity(np.zeros([num_steps_saved,3], np.float32), unit.angstroms/unit.picosecond)
    pot_en["potential_energies{0}".format(ii)] = unit.Quantity(np.zeros([num_steps_saved], np.float32), unit.kilocalories_per_mole)
    kin_en["kinetic_energies{0}".format(ii)] = unit.Quantity(np.zeros([num_steps_saved], np.float32), unit.kilocalories_per_mole)
    init_pos["initial_positions{0}".format(ii)] = np.zeros([n_particles, 3], np.float32) * unit.nanometers
    init_vel["initial_velocities{0}".format(ii)] = None 

Escribe T inicial: 300
Escribe la temperatura máxima: 305


In [4]:
contxt = []
for x in intg.values():
        contxt.append(x)

In [23]:
for cont in contxt:
    context[ii].setPositions(initial_positions[ii])
    for T in reps:
        context[ii]setVelocitiesToTemperature(T)

6


In [24]:
print(contxt)

[<simtk.openmm.openmm.LangevinIntegrator; proxy of <Swig Object of type 'OpenMM::LangevinIntegrator *' at 0x7fa7e1b059f0> >, <simtk.openmm.openmm.LangevinIntegrator; proxy of <Swig Object of type 'OpenMM::LangevinIntegrator *' at 0x7fa7d5741690> >, <simtk.openmm.openmm.LangevinIntegrator; proxy of <Swig Object of type 'OpenMM::LangevinIntegrator *' at 0x7fa7d57411e0> >, <simtk.openmm.openmm.LangevinIntegrator; proxy of <Swig Object of type 'OpenMM::LangevinIntegrator *' at 0x7fa7d5741660> >, <simtk.openmm.openmm.LangevinIntegrator; proxy of <Swig Object of type 'OpenMM::LangevinIntegrator *' at 0x7fa7d5741180> >, <simtk.openmm.openmm.LangevinIntegrator; proxy of <Swig Object of type 'OpenMM::LangevinIntegrator *' at 0x7fa7d57413f0> >]


In [ ]:
context = mm.Context(system, intg["integrator0"], platform)

contxt = []
contxt.append(context)
print(contxt[0])



In [ ]:
step_size = 0.01*unit.picoseconds
kB = unit.BOLTZMANN_CONSTANT_kB * unit.AVOGADRO_CONSTANT_NA
pressure    = None

num_steps       = 20000
saving_period   = 100
num_steps_saved = int(num_steps/saving_period)

friction   = 1.0/unit.picosecond
platform_name = 'CPU'  
platform = mm.Platform.getPlatformByName(platform_name)

integrator = mm.LangevinIntegrator(reps[0], friction, step_size)
integrator2 = mm.LangevinIntegrator(reps[1], friction, step_size)

context = mm.Context(system, integrator, platform)

In [ ]:
integrator

In [ ]:
integrator2

In [ ]:
context

In [ ]:
temps = list(range(300,306))*unit.kelvin

In [ ]:
temps